# **GluonTS**

GluonTS is a toolkit that is specifically designed for probabilistic time series modeling, It is a subpart of the Gluon organization, Gluon is an open-source deep-learning interface that allows developers to build neural nets without compromising performance and efficiency. AWS and Microsoft first introduced it on October 12th, 2017 that provides many different neural network architectures and leverages the deep learning models. It combines many packages into one like mxnet– a lightweight, portable, flexible Distributed/Mobile Deep learning model; for Python, R, Julia, Scala. Go, Javascript, and more.

# **Installation**

In [ ]:

!python -m pip install pip --upgrade --user -q --no-warn-script-location
!python -m pip install numpy pandas seaborn matplotlib scipy statsmodels sklearn tensorflow keras torch torchvision \
    tqdm scikit-image pmdarima --user -q --no-warn-script-location
!python -m pip install --upgrade mxnet~=1.7 gluonts --user -q
import IPython
IPython.Application.instance().kernel.do_shutdown(True)


# **Getting Started**

We have seen time series forecasting using TensorFlow and PyTorch, but they come with a lot of code and require great proficiency over the framework. GluonTS provide simple and on point code for running your time series forecasting here is an example code to run GluonTS for predicting Twitter volume with DeepAR.

In [ ]:
#importing gluonTS utilities and pandas
from gluonts.dataset import common
from gluonts.model import deepar
from gluonts.mx.trainer import Trainer
import pandas as pd

#getting train datatset of twitter volume
url = "https://raw.githubusercontent.com/numenta/NAB/master/data/realTweets/Twitter_volume_AMZN.csv"
df = pd.read_csv(url, header=0, index_col=0)
data = common.ListDataset([{
    "start": df.index[0],
    "target": df.value[:"2015-04-05 00:00:00"]
}],
                          freq="5min")
#initializing trainers and deepAR estimators
trainer = Trainer(epochs=10)
estimator = deepar.DeepAREstimator(
    freq="5min", prediction_length=12, trainer=trainer)
predictor = estimator.train(training_data=data)

prediction = next(predictor.predict(data))
print(prediction.mean)
prediction.plot()

Let’s take an example dataset to forecast time series dataset using GluonTS

In [ ]:
#importing modules
%matplotlib inline
import mxnet as mx
from mxnet import gluon
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

Importing inbuilt GluonTS datasets

In [ ]:
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas
print(f"Available datasets: {list(dataset_recipes.keys())}")

We are going to use the m4_hourly dataset, now the datasets provided by gluonts are objects that consist of three main attributes we will see later.

In [ ]:
dataset = get_dataset("m4_hourly", regenerate=True)

Three main attributes of GluonTs dataset are dataset.train is a training dataset, dataset.test is a testing dataset, and dataset.metadata contains metadata of the dataset. Let’s plot and iterate the dataset using the following command.

In [ ]:
entry = next(iter(dataset.train))
train_series = to_pandas(entry)
train_series.plot()
plt.grid(which="both")
plt.legend(["train series"], loc="upper left")
plt.show()

Similarly, you can plot test dataset

In [ ]:
entry = next(iter(dataset.test))
test_series = to_pandas(entry)
test_series.plot()
plt.axvline(train_series.index[-1], color='r') # end of train dataset
plt.grid(which="both")
plt.legend(["test series", "end of train series"], loc="upper left")
plt.show()

Preprocessing 

In [ ]:
N = 10  # number of time series
T = 100  # number of timesteps
prediction_length = 24
freq = "1H"
custom_dataset = np.random.normal(size=(N, T))
start = pd.Timestamp("01-01-2019", freq=freq)  # can be different for each time series

#Now for splitting dataset and converting it to gluonts format use following commands:

from gluonts.dataset.common import ListDataset
# train dataset: cut "prediction_length", add "target" and "start" fields
train_ds = ListDataset([{'target': x, 'start': start} 
                        for x in custom_dataset[:, :-prediction_length]],
                       freq=freq)
# test dataset: using whole dataset, add "target" and "start" fields
test_ds = ListDataset([{'target': x, 'start': start} 
                       for x in custom_dataset],
                      freq=freq)

### Training

GlounTS comes with its own hyper parameters and feedforward neural network like SimpleFeedForwardEstimator that accepts an input window of length context_length and predicts the distribution of the value. Let;s import necessary training methods and assign estimators values using following commands:

In [ ]:
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.mx.trainer import Trainer
estimator = SimpleFeedForwardEstimator(
    num_hidden_dimensions=[10],
    prediction_length=dataset.metadata.prediction_length,
    context_length=100,
    freq=dataset.metadata.freq,
    trainer=Trainer(ctx="cpu", 
                    epochs=5, 
                    learning_rate=1e-3, 
                    num_batches_per_epoch=100
                   )
)
#start training
predictor = estimator.train(dataset.train)

### Evaluate

For evaluating the models we have further make_evaluation_predictions function that automates the process of prediction and model evaluation.

In [ ]:
from gluonts.evaluation.backtest import make_evaluation_predictions
forecast_it, ts_it = make_evaluation_predictions(
    dataset=dataset.test,  # test dataset
    predictor=predictor,  # predictor
    num_samples=100,  # number of sample paths for evaluation
)

Convert the generators to list to ease the computations and examine the first element of these lists:

In [ ]:
forecasts = list(forecast_it)
tss = list(ts_it)
# first entry of the time series list
ts_entry = tss[0]

Convert the first five value of time-series from pandas to NumPy and initialize first entry of dataset.test

In [ ]:
np.array(ts_entry[:5]).reshape(-1,)
dataset_test_entry = next(iter(dataset.test))

Similarly first 5 values and forecast entries

In [ ]:
dataset_test_entry['target'][:5]
forecast_entry = forecasts[0]

### Output

For visualizing the outputs use following commands:

In [ ]:
def plot_prob_forecasts(ts_entry, forecast_entry):
    plot_length = 150 
    prediction_intervals = (50.0, 90.0)
    legend = ["observations", "median prediction"] + [f"{k}% prediction interval" for k in prediction_intervals][::-1]

    fig, ax = plt.subplots(1, 1, figsize=(10, 7))
    ts_entry[-plot_length:].plot(ax=ax)  # plot the time series
    forecast_entry.plot(prediction_intervals=prediction_intervals, color='g')
    plt.grid(which="both")
    plt.legend(legend, loc="upper left")
    plt.show()
plot_prob_forecasts(ts_entry, forecast_entry)

You can also evaluate the quality of time series forecast using evaluator class, that can compute the aggregate performance metrics,

In [ ]:
from gluonts.evaluation import Evaluator
evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(dataset.test))
print(json.dumps(agg_metrics, indent=4))

#**Related Articles:**

> * [Guide to Time Series Forecasting with GluonTS](https://analyticsindiamag.com/gluonts-pytorchts-for-time-series-forecasting/)

> * [Tensorflow Core](https://analyticsindiamag.com/time-series-forecasting-using-tensorflow-core/)

> * [LSTM RNN on Foreign Exchange Rate Prediction](https://analyticsindiamag.com/foreign-exchange-rate-prediction-using-lstm-recurrent-neural-network/)

> * [Pyflux](https://analyticsindiamag.com/pyflux-guide-python-library-for-time-series-analysis-and-prediction/)

> * [Atspy](https://analyticsindiamag.com/hands-on-guide-to-atspy-for-automating-the-time-series-forecasting/)

> * [AutoTS](https://analyticsindiamag.com/hands-on-guide-to-autots-effective-model-selection-for-multiple-time-series/)